random forest traning model for AM-I, AM-II, AM-III

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import json
from datetime import datetime
from glob import glob
from tqdm import tqdm
import optuna
from sklearn.model_selection import KFold, learning_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# ===================== Color Scheme =====================
IPHONE_COLORS = {
    'scatter': '#007AFF',
    'line':    '#AEAEB2',
    'text':    '#000000'
}

# ===================== Random Seed =====================
def set_seed(seed=42):
    np.random.seed(seed)
set_seed(42)

# ===================== Path Configuration =====================
data_dir = './train_test_split'
OUTPUT_FOLDER = './'
model_dir = os.path.join(OUTPUT_FOLDER, 'RT-models')
os.makedirs(model_dir, exist_ok=True)

# ===================== Features and Target =====================
feature_columns = ['MolWt', 'logP', 'TPSA', 'H_bond_donors', 'H_bond_acceptors']
morgan_fp_columns = [f'fp_{i}' for i in range(1024)]
fp_columns = [f'col{i}' for i in range(823)]
all_feature_columns = feature_columns + morgan_fp_columns + fp_columns
target_column = 'UV_RT-s'

# ===================== Plotting Functions =====================
def plot_scatter_and_residuals(y_true, y_pred, base_name, save_dir):
    # Scatter plot
    plt.figure(figsize=(6,6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=2)
    for spine in ['top','right','bottom','left']:
        ax.spines[spine].set_visible(True)
    plt.grid(False)
    
    plt.scatter(y_true, y_pred, alpha=0.8, s=70, color=IPHONE_COLORS['scatter'])
    lims = [min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())]
    plt.plot(lims, lims, linestyle='--', color=IPHONE_COLORS['line'], linewidth=3)

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    plt.xlabel("True Retention Time (s)", fontsize=17)
    plt.ylabel("Predicted Retention Time (s)", fontsize=17)
    plt.text(0.05, 0.95, f"R² = {r2:.3g}\nMAE = {mae:.3g}", transform=ax.transAxes, va='top', fontsize=16, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    scatter_path = os.path.join(save_dir, f"scatter.png")
    plt.savefig(scatter_path, dpi=600)
    plt.close()

    # Residual plot
    residuals = y_pred - y_true
    plt.figure(figsize=(6,6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=2)
    for spine in ['top','right','bottom','left']:
        ax.spines[spine].set_visible(True)
    plt.grid(False)
    
    plt.scatter(y_pred, residuals, alpha=0.8, s=70, color=IPHONE_COLORS['scatter'])
    plt.axhline(y=0, linestyle='--', color=IPHONE_COLORS['line'], linewidth=3)

    plt.xlabel("Predicted Retention Time (s)", fontsize=17)
    plt.ylabel("Residuals (Predicted - True)", fontsize=17)
    plt.text(0.5, -0.15, "Residual Plot", ha='center', va='center', transform=ax.transAxes, fontsize=16, color=IPHONE_COLORS['text'])
    plt.text(0.05, 0.95, f"R² = {r2:.3g}\nMAE = {mae:.3g}", transform=ax.transAxes, va='top', fontsize=16, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    residual_path = os.path.join(save_dir, f"residuals.png")
    plt.savefig(residual_path, dpi=600)
    plt.close()
    
    return scatter_path, residual_path

# ===================== Training Pipeline =====================
all_metrics = []
train_files = sorted(glob(os.path.join(data_dir, '*_train.csv')))

for train_file in tqdm(train_files, desc="Training Sets"):
    base = os.path.basename(train_file).replace('_train.csv','')
    test_file = train_file.replace('_train.csv','_test.csv')
    if not os.path.exists(test_file):
        print(f"[Skip] {test_file} not found")
        continue

    print(f"\n🚀 Processing: {base}")
    
    # ===================== Create Dataset-specific Directory =====================
    dataset_dir = os.path.join(model_dir, base)
    os.makedirs(dataset_dir, exist_ok=True)
    print(f"  → Output directory: {dataset_dir}")
    
    # ===================== Load Data =====================
    tr = pd.read_csv(train_file).dropna()
    te = pd.read_csv(test_file).dropna()
    X_train, y_train = tr[all_feature_columns].values, tr[target_column].values
    X_test, y_test = te[all_feature_columns].values, te[target_column].values
    
    # Save dataset info
    dataset_info = {
        'dataset_name': base,
        'train_samples': len(tr),
        'test_samples': len(te),
        'features': all_feature_columns,
        'target': target_column,
        'feature_count': len(all_feature_columns)
    }
    
    with open(os.path.join(dataset_dir, 'dataset_info.json'), 'w') as f:
        json.dump(dataset_info, f, indent=2)

    # ===================== Optuna Hyperparameter Tuning =====================
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 10, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
        }
        rf = RandomForestRegressor(random_state=42, **params, n_jobs=26)
        fold_r2 = []
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for tr_idx, va_idx in kf.split(X_train):
            rf.fit(X_train[tr_idx], y_train[tr_idx])
            y_pred_fold = rf.predict(X_train[va_idx])
            fold_r2.append(r2_score(y_train[va_idx], y_pred_fold))
        return np.mean(fold_r2)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=30)
    best_params = study.best_params
    
    # Save optimization study
    optuna_results = {
        'best_params': best_params,
        'best_value': study.best_value,
        'n_trials': len(study.trials)
    }
    
    with open(os.path.join(dataset_dir, 'optuna_results.json'), 'w') as f:
        json.dump(optuna_results, f, indent=2)
    
    print(f"  → Best Parameters: {best_params}")

    # ===================== Train Final Model =====================
    rf_final = RandomForestRegressor(random_state=42, **best_params, n_jobs=26)
    rf_final.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = rf_final.predict(X_train)
    y_test_pred = rf_final.predict(X_test)

    # Calculate metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_rmse = np.sqrt(train_mse)
    
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = np.sqrt(test_mse)

    # ===================== Learning Curve =====================
    ts, tr_sc, vl_sc = learning_curve(
        rf_final, X_train, y_train,
        cv=KFold(n_splits=5, shuffle=True, random_state=42),
        scoring='neg_mean_squared_error',
        train_sizes=np.linspace(0.1, 1.0, 10),
        n_jobs=26
    )
    tr_mean = -np.mean(tr_sc, axis=1)
    vl_mean = -np.mean(vl_sc, axis=1)
    plt.figure(figsize=(10, 6))
    plt.plot(ts, tr_mean, 'o-', label='Train MSE')
    plt.plot(ts, vl_mean, 'o-', label='Val MSE')
    plt.xlabel('Training Size')
    plt.ylabel('MSE')
    plt.title(f'Learning Curve: {base}')
    plt.legend()
    plt.grid(True)
    lc_path = os.path.join(dataset_dir, "learning_curve.png")
    plt.savefig(lc_path, dpi=600)
    plt.close()

    # ===================== Generate Scatter and Residual Plots =====================
    scatter_path, residual_path = plot_scatter_and_residuals(y_test, y_test_pred, base, dataset_dir)

    # ===================== Save Model =====================
    model_path = os.path.join(dataset_dir, "random_forest_model.pkl")
    joblib.dump(rf_final, model_path)
    
    # Save model info
    model_info = {
        'model_type': 'RandomForestRegressor',
        'model_path': "random_forest_model.pkl",
        'feature_importance': dict(zip(all_feature_columns, rf_final.feature_importances_.tolist())),
        'best_params': best_params
    }
    
    with open(os.path.join(dataset_dir, 'model_info.json'), 'w') as f:
        json.dump(model_info, f, indent=2)

    # ===================== Save Predictions =====================
    # Save training set predictions
    train_pred_df = tr.copy()
    train_pred_df['pred_RT'] = y_train_pred
    train_pred_df['residual'] = y_train_pred - y_train
    train_pred_df.to_csv(os.path.join(dataset_dir, "train_predictions.csv"), index=False)
    
    # Save test set predictions
    test_pred_df = te.copy()
    test_pred_df['pred_RT'] = y_test_pred
    test_pred_df['residual'] = y_test_pred - y_test
    test_pred_df.to_csv(os.path.join(dataset_dir, "test_predictions.csv"), index=False)

    # ===================== Save Performance Metrics =====================
    performance_metrics = {
        'training': {
            'MSE': float(train_mse),
            'RMSE': float(train_rmse),
            'R2': float(train_r2),
            'MAE': float(train_mae)
        },
        'testing': {
            'MSE': float(test_mse),
            'RMSE': float(test_rmse),
            'R2': float(test_r2),
            'MAE': float(test_mae)
        }
    }
    
    with open(os.path.join(dataset_dir, 'performance_metrics.json'), 'w') as f:
        json.dump(performance_metrics, f, indent=2)
    
    # For summary CSV
    metrics = {
        "Dataset": base,
        "Train_Samples": len(tr),
        "Test_Samples": len(te),
        "Train_R2": train_r2,
        "Train_MAE": train_mae,
        "Test_R2": test_r2,
        "Test_MAE": test_mae,
        "Test_MSE": test_mse,
        "Test_RMSE": test_rmse,
        "Best_n_estimators": best_params['n_estimators'],
        "Best_max_depth": best_params['max_depth'],
        "Dataset_Directory": dataset_dir,
        "Model_File": "random_forest_model.pkl"
    }
    all_metrics.append(metrics)
    
    print(f"  → Training R²: {train_r2:.4f}, MAE: {train_mae:.2f}")
    print(f"  → Testing R²: {test_r2:.4f}, MAE: {test_mae:.2f}")
    print(f"  → All results saved to: {dataset_dir}")

# ===================== Save Global Summary =====================
summary_df = pd.DataFrame(all_metrics)
summary_sorted = summary_df.sort_values(by="Test_R2", ascending=False)
summary_path = os.path.join(model_dir, "summary_all_datasets.csv")
summary_sorted.to_csv(summary_path, index=False)

# Save summary as JSON as well
summary_json = summary_sorted.to_dict(orient='records')
with open(os.path.join(model_dir, "summary_all_datasets.json"), 'w') as f:
    json.dump(summary_json, f, indent=2)

/home/xuxianyan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Training Sets:   0%|          | 0/3 [00:00<?, ?it/s]


🚀 Processing: AM-I-filtered_with_labels_k4
  → Output directory: ./RT-models/AM-I-filtered_with_labels_k4


[I 2026-01-19 09:30:06,954] A new study created in memory with name: no-name-38a7295b-3119-4d1b-b06c-645a74cebaf7
[I 2026-01-19 09:30:21,886] Trial 0 finished with value: 0.7222546977320181 and parameters: {'n_estimators': 443, 'max_depth': 13, 'min_samples_split': 13, 'min_samples_leaf': 5, 'max_features': None, 'bootstrap': True}. Best is trial 0 with value: 0.7222546977320181.
[I 2026-01-19 09:30:27,645] Trial 1 finished with value: 0.22832218702381554 and parameters: {'n_estimators': 282, 'max_depth': 11, 'min_samples_split': 11, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 0.7222546977320181.
[I 2026-01-19 09:30:48,008] Trial 2 finished with value: 0.5738155131325234 and parameters: {'n_estimators': 362, 'max_depth': 50, 'min_samples_split': 4, 'min_samples_leaf': 8, 'max_features': None, 'bootstrap': False}. Best is trial 0 with value: 0.7222546977320181.
[I 2026-01-19 09:31:03,058] Trial 3 finished with value: 0.5788516380257639 

  → Best Parameters: {'n_estimators': 164, 'max_depth': 33, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': True}


Training Sets:  33%|███▎      | 1/3 [05:08<10:17, 308.89s/it]

...

  → Training R²: 0.9373, MAE: 1.69
  → Testing R²: 0.7293, MAE: 3.50
  → All results saved to: ./RT-models/AM-II-filtered_with_labels_k3

🚀 Processing: AM-III-filtered_with_labels_k4
  → Output directory: ./RT-models/AM-III-filtered_with_labels_k4


[I 2026-01-19 09:38:14,913] Trial 0 finished with value: 0.7107716692343702 and parameters: {'n_estimators': 140, 'max_depth': 48, 'min_samples_split': 11, 'min_samples_leaf': 8, 'max_features': None, 'bootstrap': True}. Best is trial 0 with value: 0.7107716692343702.
[I 2026-01-19 09:38:18,910] Trial 1 finished with value: 0.5635809625275275 and parameters: {'n_estimators': 382, 'max_depth': 46, 'min_samples_split': 14, 'min_samples_leaf': 5, 'max_features': None, 'bootstrap': False}. Best is trial 0 with value: 0.7107716692343702.
[I 2026-01-19 09:38:21,314] Trial 2 finished with value: 0.5249734897148249 and parameters: {'n_estimators': 124, 'max_depth': 26, 'min_samples_split': 8, 'min_samples_leaf': 7, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 0 with value: 0.7107716692343702.
[I 2026-01-19 09:38:24,843] Trial 3 finished with value: 0.6997658038667949 and parameters: {'n_estimators': 350, 'max_depth': 42, 'min_samples_split': 3, 'min_samples_leaf': 10, 'max_feature

  → Best Parameters: {'n_estimators': 187, 'max_depth': 28, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': None, 'bootstrap': True}


Training Sets: 100%|██████████| 3/3 [10:00<00:00, 200.21s/it]

  → Training R²: 0.9545, MAE: 2.20
  → Testing R²: 0.7772, MAE: 5.15
  → All results saved to: ./RT-models/AM-III-filtered_with_labels_k4
